In [1]:
import numpy as np
import scipy.linalg
import functools
!pip install cvxpy
!pip install cvxopt
!pip install cvxpy-base
import cvxpy as cp
inv = scipy.linalg.inv 
randn = np.random.randn
from numpy.linalg import matrix_rank
from scipy.linalg import logm, expm
from scipy.linalg import sqrtm
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import math
from cvxpy.atoms.axis_atom import AxisAtom
from cvxpy.atoms.norm1 import norm1
from cvxpy.atoms.norm_inf import norm_inf
from cvxpy.constraints.constraint import Constraint
from cvxpy.utilities.power_tools import pow_high, pow_mid, pow_neg
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import rc
import pickle
import scipy
from scipy.linalg import expm, sinm, cosm, logm
from scipy.stats import unitary_group
from scipy import random
from scipy.stats import multivariate_normal
!pip install GMM
!pip install MKLpy
!pip install algorithms
!sudo sh -c 'echo 1 >/proc/sys/kernel/perf_event_paranoid'
from gmm import gmm
import timeit
import tracemalloc
import sklearn
from sklearn.datasets import make_spd_matrix
### Channel Matrices
power_delay_profile = np.array([0, -1 ,-9, -10, -15, -20]) # ITU_Vehicular_A channel #### attenuation over differnt paths
delay_profile = 1e-2*np.array([0 ,300 ,700 ,1100 ,1700, 2500])  #### time delay in multiple paths

You should consider upgrading via the 'c:\users\dell\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\dell\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\dell\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\dell\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\dell\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\dell\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.
The system cannot find the path specified.


In [2]:
### Consider 'mXm MIMO' system
m = 2

### Transmit Antennas
Nt = m

### Receive Antennas
Nr = m

## Number of Subcarriers
Nsc = 8

channel_samples = 100000

# **Definitions**

In [3]:
def poly_add(m,p1,p2):
    l1 = p1.shape[0]
    l2 = p2.shape[0]
    if(l1 == l2):
        return p1 + p2
    elif(l1 > l2):
        p2_mod = np.zeros(p1.shape,dtype = np.complex128)
        p2_mod[l1-l2:l1] = p2
        return p1 + p2_mod
    else:
        p1_mod = np.zeros(p2.shape,dtype = np.complex128)
        p1_mod[l2-l1:l2] = p1
        return p1_mod + p2

### computing the coefficients of product of two polynomial (as a convolution of their coefficient vectors) 
def poly_convolve(m,p1,p2):
    if(m == 1):
        return np.convolve(p1,p2)
    else:
        result = []
        l1 = p1.shape[0]
        l2 = p2.shape[0]
        p1_mod = np.zeros((l1+2*(l2-1),m,m), dtype = np.complex128)# append zeroes

        #### zero padded p1
        for i in range(l1 + l2 - 1):
            if (i < l2 -1):
                p1_mod[i] = 0
            elif (i < l1 + l2 - 1):
                p1_mod[i] = p1[i - (l2 - 1)]
            else :
                p1_mod[i] = 0
                
        ### convlution of p1 & p2
        for i in range(l1 + l2 - 1):
            temp = np.zeros((m,m), dtype = np.complex128)
            for j in range(l2):
                temp = temp + p1_mod[i+j]@p2[l2-1-j]
            result.append(temp)
        return np.array(result)


### Evaluating the polynomial matrix at a particular frequency
def poly_eval(m,p,omega):
    l = p.shape[0]
    ans = np.zeros((m,m),dtype = np.complex128)
    s = omega
    for i in range(l):
        ans = ans + p[l-1-i]*(s**i)
    return ans

In [4]:
### Base step in induaction based solution construction
def base_interpolation_poly(m,omega,U,Gamma):
    I = np.eye(m)
    N = [(I+ (U @ inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) ) ,(-1*omega*I+ (U @ inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) )]
    D = [(I+ (inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) )     ,   (-1*omega*I+ (inv(Gamma) @ (I -np.conj(U.T))) * omega /(omega+1) ) ]
    return np.array(N),np.array(D);

In [5]:
### Matrix used to modify the data in inductive step of Induction based solution construction 
def Transformation_mat_poly(m,omega_1,U1,Gamma_1): 
    I = np.eye(m)
    H = [np.array([2*I - (omega_1+1)*inv(Gamma_1), -2 * omega_1 * I - (omega_1 + 1)*inv(Gamma_1)]),
         np.array([(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T ,(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T]),
         np.array([(omega_1 + 1)*-1*U1 @ inv(Gamma_1),  (omega_1 + 1)*-1*U1 @ inv(Gamma_1)]),
         np.array([2*I + ((omega_1 + 1)* U1 @ inv(Gamma_1) @ U1.conj().T)   , (-2 * omega_1 * I) + ((omega_1 + 1) * U1 @ inv(Gamma_1) @ U1.conj().T) ])]
    return H

In [6]:
### Final step of combining the solutions (of N-1 points & 1 point) in Induction based solution construction {NOTE: This is done iteratively} 
def final_interpolation_poly(m,omega_1,U1,Gamma_1,Ns_hat,Ds_hat):
    I = np.eye(m)
    D1 = [2*I + (omega_1+1)*inv(Gamma_1),    -2 * omega_1 * I + (omega_1 + 1)*inv(Gamma_1)]
    D2 = [-1*(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T ,  -1*(omega_1 + 1) *inv(Gamma_1) @ U1.conj().T]
    N1 = [2*I - (omega_1 + 1) *U1 @ inv(Gamma_1) @ U1.conj().T   , -2 * omega_1 * I - (omega_1 + 1) * U1 @ inv(Gamma_1) @ U1.conj().T]
    N2 = [(omega_1 + 1)*U1 @ inv(Gamma_1),  (omega_1 + 1)*U1 @ inv(Gamma_1)]

    Dz = poly_add(m,poly_convolve(m,np.array(D1),Ds_hat),poly_convolve(m,np.array(D2),Ns_hat))
    Nz = poly_add(m,poly_convolve(m,np.array(N1),Ns_hat),poly_convolve(m,np.array(N2),Ds_hat))
    return Nz, Dz;

In [7]:
def Recursive_interpolation_poly(n,m,omega_array,U_array,Gamma_array):
    I = np.eye(m)
    if(n != omega_array.shape[0]) or (n != U_array.shape[0]) or (n != Gamma_array.shape[0]):
        print("Incorrect no. of points in recursion")
        return
        
    if(n == 1): # Base step
        Ns_hat, Ds_hat = base_interpolation_poly(m,omega_array[0],U_array[0],Gamma_array[0])
        return Ns_hat, Ds_hat
    
    else: ### modify the data 
        omega = omega_array[0]
        U = U_array[0]
        Gamma = Gamma_array[0]

        ### modification for next step
        omega_array_mod = omega_array[1:n]
        eval_H = Transformation_mat_poly(m,omega,U,Gamma)
        U_array_mod = np.zeros((n-1,m,m), dtype = np.complex128)

        for i in range(n-1):
            ### Converting to polynomial
            H_omega_0 = poly_eval(m,eval_H[0],omega_array[i+1])
            H_omega_1 = poly_eval(m,eval_H[1],omega_array[i+1])
            H_omega_2 = poly_eval(m,eval_H[2],omega_array[i+1])
            H_omega_3 = poly_eval(m,eval_H[3],omega_array[i+1])        

            #### Modifying the data 'B hat' in PPT  
            V_hat_1 = H_omega_0 + H_omega_1 @ U_array[i+1]
            V_hat_2 = H_omega_2 + H_omega_3 @ U_array[i+1]

            U_array_mod[i] =  V_hat_2 @ inv(V_hat_1)    

        Gamma_array_mod = np.zeros((n-1,m,m), dtype = np.complex128)
        
        for i in range(n-1):
            Gamma_array_mod[i] = Gamma_array[i+1] - (((I - np.conj(U_array[i+1].T)@U)@inv(Gamma)@(I - np.conj(U.T)@U_array[i+1]))/((1- (omega_array[i+1]/omega)) * (1- (omega/omega_array[i+1])) ))
        
        ### recursive
        Ns_hat, Ds_hat = Recursive_interpolation_poly(n-1,m,omega_array_mod,U_array_mod,Gamma_array_mod)
        
        ### combining
        Nz, Dz = final_interpolation_poly(m,omega,U,Gamma,Ns_hat,Ds_hat)
        return Nz, Dz

def Gs_eval(m,Ns,Ds,omega):
    return poly_eval(m,Ns,omega)@inv(poly_eval(m,Ds,omega))

In [8]:
def SNIP_boundary(m,n,a,power_delay_profile, delay_profile,channel_cont_omega,Flag,U_array_ip,omega_array_ip,scale):
  Gamma_array_ip = np.zeros((n,m,m), dtype = complex)
  delta_w = 10**(-12)
  for i in range(n):
    U,S,Vh = scipy.linalg.svd(channel_cont_omega(np.imag(np.log(omega_array_ip[i])) + delta_w))
    U_w_plus_delta = U
    Phi_w_plus_delta = -1j*logm(U_w_plus_delta)
    Phi_w_plus_delta_Her = (Phi_w_plus_delta + np.conj(Phi_w_plus_delta.T))/2  

    U_w = U_array_ip[i]

    Phi_w = -1j*logm(U_w) ##### Phase ???
    Phi_w_Her = (Phi_w + np.conj(Phi_w.T))/2

    ### since delta_w is already scaled, we have to un-scale it before deividing below
    delta_w = delta_w*scale
    differentiate_phi = (Phi_w_plus_delta_Her - Phi_w_Her)/delta_w  #### differential of phase

    Gamma = 2*sqrtm(np.conj(differentiate_phi.T)@differentiate_phi) ## square root of matrix product
    gamma_her = np.conj(Gamma.T) - Gamma

    Gamma_array_ip[i] = Gamma
    Group_delays = Gamma_array_ip
  ### We want to compute the interpolation at original omega values(i.e unscaled) , so we have to unscale the omega values before passing them through Recursive Interpolation
  for q in range(len(omega_array_ip)):
    omega_array_ip[q] = np.exp((scale)*np.log(omega_array_ip[q]))

  Nz, Dz = Recursive_interpolation_poly(n,m,omega_array_ip,U_array_ip,Gamma_array_ip)
  return Nz,Dz

In [9]:
def tracing_start():
    tracemalloc.stop()
    # print("nTracing Status : ", tracemalloc.is_tracing())
    tracemalloc.start()
    # print("Tracing Status : ", tracemalloc.is_tracing())
def tracing_mem():
    first_size, first_peak = tracemalloc.get_traced_memory()
    peak = first_peak/(1024*1024)
    return(peak)
    # print("Peak Size in MB - ", peak)

In [10]:
def gparam_test(U):
    M = np.size(U,1)
    U_prev = U
    phi_vals = np.zeros(((M*(M+1))//2))
    theta_vals = np.zeros(((M*(M-1))//2))
    current_column_angles = []
    start = 0
    start2 = 0
    for i in range(M):
        current_column_angles = np.concatenate((np.zeros((i)), np.angle(U_prev[i:,i]).T))#,axis = 1)
        U_current_column_real = np.diag(np.exp(-1j * current_column_angles)) @ U_prev
        phi_vals[start:start+M-i] = np.angle(U_prev[i:,i]).T
        for j in range(i+1,M):
            G = np.eye(M)
            if not np.allclose(np.real(U_current_column_real[j,i]),U_current_column_real[j,i]):
                print("real column error")
                print(U_current_column_real[j,i])
            theta = np.arctan2(np.real(U_current_column_real[j,i]),np.real(U_current_column_real[i,i]))
            G[i, i] = np.cos(theta)
            G[i, j] = np.sin(theta)
            G[j, i] = -1*np.sin(theta)
            G[j, j] = np.cos(theta)
            theta_vals[start2+(j-i-1)] = theta
            U_current_column_real = G@U_current_column_real
        start = start + (M-i)
        start2 = start2 + (M-i-1)
        U_prev = U_current_column_real
    return [phi_vals, theta_vals]
    

In [11]:
def givens_reconstruct(phi_theta_vals):
    M = int(np.sqrt(len(phi_theta_vals)))
    phi_vals = phi_theta_vals[0:(M*(M+1)//2)]
    phi_vals = phi_vals[::-1]
    theta_vals = phi_theta_vals[(M*(M+1)//2):]
    theta_vals = theta_vals[::-1]
    U = np.eye(M)
    for i in reversed(range(M)):
        for j in reversed(range(i+1,M)):
            G = np.eye(M)
            theta = theta_vals[0]
            theta_vals = theta_vals[1:]
            G[np.ix_([i,j],[i,j])] = np.array([[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]])
            U = G.T @ U
        i_complement = M - i 
        if (i == M-1):
            phi_end = -(M*(M+1)//2)
        else:
            y = i_complement
            phi_end = y*(y+1)//2 - 1
        phi_start = phi_end - i_complement
        current_phi_vals = np.r_[np.zeros(i),phi_vals[phi_end:phi_start:-1]]
        U = np.diag(np.exp(1j * current_phi_vals)) @ U
    return U

In [12]:
def get_computational_complexity(method,m,Nsc,channel_samples,power_delay_profile,delay_profile):
    average_achievable_rate = 0
    times= 0
    counts = 0
    snip_time= 0
    methods = ["SNIP based Filtering", "GEODESIC Interpolation", "Givens Rotation"]
    for q in range(channel_samples):
        n = 2 ### no.of data points
        H = np.zeros((5,m,m),dtype = complex)
        for i in range(5):
          H[i] = (randn(m,m) + 1j * randn(m,m))*(1/np.sqrt(2))
        channel_cont_omega = lambda omega : (H[0])*(10**(power_delay_profile[0]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[0])) +  (H[1])*(10**(power_delay_profile[1]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[1])) + (H[2])*(10**(power_delay_profile[2]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[2])) +  (H[3])*(10**(power_delay_profile[3]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[3])) + (H[4])*(10**(power_delay_profile[4]/20))*(np.exp(-1j*2*np.pi*omega*delay_profile[4]))

        #### Subcarrier frequencies
        freq = np.array([np.exp(1j*(i+1)/(Nsc+1)) for i in range(Nsc)])

        channel = [channel_cont_omega(-1j*np.log(i)) for i in freq]

        precoders =[]
        for j in range(len(channel)):
          U,S,Vh = scipy.linalg.svd(channel[j])
          precoders.append(np.conj(Vh.T))

        a = 0.49

        ## SNIP Interpolation code , given the data-set
        prec = np.array([precoders[int((Nsc/n)*t)] for t in range(n)])
        inter_freq = np.array([freq[int((Nsc/n)*t)] for t in range(n)])

        if method == 0: ### SNIP
            snip_begin = timeit.default_timer()
            Nz,Dz = SNIP_boundary(m,n,a,power_delay_profile, delay_profile,channel_cont_omega,0,prec,inter_freq,1.0)
            snip_end = timeit.default_timer()
            freq_int = freq[1:-1]
            F = [Gs_eval(m,Nz,Dz,freq[0])]
            
            ### TIME
            start = timeit.default_timer()
            F1 = [Gs_eval(m,Nz,Dz,f) for f in freq_int]
            end = timeit.default_timer()

            ### MEMORY
            tracing_start()
            F1 = [Gs_eval(m,Nz,Dz,f) for f in freq_int]
            stops  = tracing_mem()
            F = F + F1 + [Gs_eval(m,Nz,Dz,freq[-1])]

            snip_time = snip_time +  snip_end - snip_begin
            times = times + end-start 
            counts = counts + stops

        elif method == 1: ### Geodesic
            b = 1
            omega_1 = -1j*np.log(freq[0])
            omega_2 = -1j*np.log(freq[-1])
            V2 = precoders[-1]
            V1 = precoders[0]
            log_freq = np.log(freq)
            diff = omega_2 - omega_1
            precoders_interp = np.zeros((Nsc-2,m,m),dtype = complex)

            ### TIME 
            start = timeit.default_timer()
            B = logm(np.conj(V1.T)@V2)
            for b in range(Nsc-2):
              t =  ((-1j*log_freq[b+1]-omega_1)/diff)/((omega_2 +1j*log_freq[b+1])/diff)
              precoders_interp[b] = V1@expm(t*B)
            end = timeit.default_timer()

            ### MEMORY
            tracing_start()
            B = logm(np.conj(V1.T)@V2)
            for b in range(Nsc-2):
              t =  ((-1j*log_freq[b+1]-omega_1)/diff)/((omega_2 +1j*log_freq[b+1])/diff)
              precoders_interp[b] = V1@expm(t*B)
            stops = tracing_mem()


            F = precoders
            F[1:Nsc-1] = precoders_interp
            times = times + end-start 
            counts = counts + stops

        elif method == 2: ### Givens Rotation
            b = 1
            omega_1 = -1j*np.log(freq[0])
            omega_2 = -1j*np.log(freq[-1])
            V2 = precoders[-1]
            V1 = precoders[0]
            log_freq = np.log(freq)
            diff = omega_2 - omega_1
            precoders_interp = np.zeros((Nsc-2,m,m),dtype = complex)

            phi_vals_0, theta_vals_0 = gparam_test(V1)
            phi_theta_vals_0 = np.zeros(len(phi_vals_0) + len(theta_vals_0))
            phi_theta_vals_0[:len(phi_vals_0)] = phi_vals_0
            phi_theta_vals_0[len(phi_vals_0):] = theta_vals_0
            phi_vals_1, theta_vals_1 = gparam_test(V2)
            phi_theta_vals_1 = np.zeros(len(phi_vals_1) + len(theta_vals_1))
            phi_theta_vals_1[:len(phi_vals_1)] = phi_vals_1
            phi_theta_vals_1[len(phi_vals_1):] = theta_vals_1

            ### TIME 
            B = logm(np.conj(V1.T)@V2)

            phi_theta_vals = ((omega_2+1j*log_freq[b+1])/(diff)*phi_theta_vals_0) + ((-1j*log_freq[b+1]-omega_1)/(diff)*phi_theta_vals_1)
            phi_vals = phi_theta_vals[0:(m*(m+1)//2)]
            phi_vals = phi_vals[::-1]
            theta_vals = phi_theta_vals[(m*(m+1)//2):]
            theta_vals = theta_vals[::-1]

            start = timeit.default_timer()
            for b in range(Nsc-2):
              # precoders_interp[b] = givens_reconstruct(((omega_2+1j*log_freq[b+1])/(diff)*phi_theta_vals_0) + ((-1j*log_freq[b+1]-omega_1)/(diff)*phi_theta_vals_1))
              U = np.eye(m)
              for i in reversed(range(m)):
                  for j in reversed(range(i+1,m)):
                      G = np.eye(m)
                      theta = theta_vals[0]
                      # theta_vals = theta_vals[1:]
                      G[np.ix_([i,j],[i,j])] = np.array([[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]])
                      U = G.T @ U
                  i_complement = m - i 
                  if (i == m-1):
                      phi_end = -(m*(m+1)//2)
                  else:
                      y = i_complement
                      phi_end = y*(y+1)//2 - 1
                  phi_start = phi_end - i_complement
                  current_phi_vals = np.r_[np.zeros(i),phi_vals[phi_end:phi_start:-1]]
                  U = np.diag(np.exp(1j * current_phi_vals)) @ U
            end = timeit.default_timer()
            precoders_interp[b] = U
            


            ### MEMORY
            tracing_start()
            B = logm(np.conj(V1.T)@V2)
            for b in range(Nsc-2):
              precoders_interp[b] = givens_reconstruct(((omega_2+1j*log_freq[b+1])/(diff)*phi_theta_vals_0) + ((-1j*log_freq[b+1]-omega_1)/(diff)*phi_theta_vals_1))
            stops = tracing_mem()
            F = precoders
            F[1:Nsc-1] = precoders_interp
            times = times + end-start 
            counts = counts + stops
    print(str(m)+"X"+str(m)+" MIMO -- "+"Method - "+str(methods[method])+" --> ","Time to construct Precoder is --> "+str(times/(channel_samples)))
    print(str(m)+"X"+str(m)+" MIMO -- "+"Method - "+str(methods[method])+" --> ","Memory consumed (Peak size) in MB --> "+str(counts/(channel_samples)))
    # if method == 0:
    #   print(str(m)+"X"+str(m)+" MIMO -- "+"Method - "+str(methods[method])+" --> ","Time to construct MATRIX ALL-PASS FILTER IS --> "+str(snip_time/(channel_samples)))
    return times/(channel_samples) , snip_time/(channel_samples)

# **COMPUTATION COMPLEXITY EVALUATION**

In [14]:
for m in range(8):
  m = m+2
  Nt = m
  Nr = m
  t2,s = get_computational_complexity(1,m,Nsc,channel_samples,power_delay_profile,delay_profile)
  t3,s = get_computational_complexity(2,m,Nsc,channel_samples,power_delay_profile,delay_profile)
  t1,s = get_computational_complexity(0,m,Nsc,channel_samples,power_delay_profile,delay_profile)

  print('-------------------------------------------------------------------------------------')


2X2 MIMO -- Method - GEODESIC Interpolation -->  Time to construct Precoder is --> 0.00610491729999967
2X2 MIMO -- Method - GEODESIC Interpolation -->  Memory consumed (Peak size) in MB --> 0.006646463394165039


C:\Users\DELL\AppData\Local\Temp\ipykernel_11824\1702866097.py:121: ComplexWarning: Casting complex values to real discards the imaginary part
  G[np.ix_([i,j],[i,j])] = np.array([[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]])
C:\Users\DELL\AppData\Local\Temp\ipykernel_11824\2393489533.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  G[np.ix_([i,j],[i,j])] = np.array([[np.cos(theta),np.sin(theta)],[-np.sin(theta),np.cos(theta)]])


2X2 MIMO -- Method - Givens Rotation -->  Time to construct Precoder is --> 0.0015773020000017367
2X2 MIMO -- Method - Givens Rotation -->  Memory consumed (Peak size) in MB --> 0.006899402618408203
2X2 MIMO -- Method - SNIP based Filtering -->  Time to construct Precoder is --> 0.0004981135999997833
2X2 MIMO -- Method - SNIP based Filtering -->  Memory consumed (Peak size) in MB --> 0.00325811767578125
-------------------------------------------------------------------------------------
3X3 MIMO -- Method - GEODESIC Interpolation -->  Time to construct Precoder is --> 0.008351216500001328
3X3 MIMO -- Method - GEODESIC Interpolation -->  Memory consumed (Peak size) in MB --> 0.007273948669433593
3X3 MIMO -- Method - Givens Rotation -->  Time to construct Precoder is --> 0.002895880799998622
3X3 MIMO -- Method - Givens Rotation -->  Memory consumed (Peak size) in MB --> 0.0076479921340942385
3X3 MIMO -- Method - SNIP based Filtering -->  Time to construct Precoder is --> 0.0004783162999